In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
from IPython.display import display
import warnings

# 경고 메시지 무시
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
def preprocess_data(df):
    # 1. '지점명'별로 그룹화하여, '일시'가 11개 이상인 지점만 필터링
    df_clean = df.groupby('지점명').filter(lambda x: len(x) >= 11)
    
    # 2. 각 지점에 대해 누락된 '일시' 값을 찾아서 추가하기
    for station in df_clean['지점명'].unique():
        station_data = df_clean[df_clean['지점명'] == station]

        # 지점명에 따른 지점
        station_number = df_clean[df_clean['지점명'] == station]['지점'].iloc[0]
        
        # '2023-11'부터 '2024-10'까지의 월별 범위
        missing_months = pd.date_range(start='2023-11-01', end='2024-10-01', freq='MS')
        # 현재 지점에 없는 월
        missing_months = missing_months[~missing_months.isin(station_data['일시'])]
        
        if len(missing_months) > 0:
            # 해당 지점의 월별 평균 계산
            monthly_mean = station_data['월 미세먼지 농도(㎍/㎥)'].mean()
            
            # 새로운 행 추가
            new_rows = pd.DataFrame({
                '지잠' : [station_number],
                '지점명': [station],
                '일시': missing_months.strftime('%Y-%m'),
                '월 미세먼지 농도(㎍/㎥)': [monthly_mean],
            })
            
            # 기존 데이터에 새 행 추가
            df_clean = pd.concat([df_clean, new_rows], ignore_index=True)

    # 3. NaN이 있는 '일시'가 11개인 지점들은 삭제
    df_clean = df_clean.groupby('지점명').filter(lambda x: len(x) == 12)

    return df_clean



In [27]:
def drawGraph(df):
   # 중복 제거한 지점 목록을 추출하여 정렬
    unique_locations = sorted(set(df['지점명']))

    # 중복 제거한 지점 목록 출력
    print("사용 가능한 지점 목록:")
    for i, name in enumerate(unique_locations, 1):
        print(f"{i}. {name}")

    # 사용자에게 몇 개의 지점을 선택할지 입력받기 (1~5 사이의 숫자)
    num_locations = int(input("선택할 지점의 개수를 입력하세요 (1~5): "))
    while num_locations < 1 or num_locations > 5:
        print("지원하지 않는 범위입니다.")
        print("1에서 5 사이의 숫자를 입력해주세요.")
        num_locations = int(input("선택할 지점의 개수를 입력하세요 (1~5): "))

    # 사용자에게 선택할 지점 번호 입력받기
    selected_indices = []
    print(f"{num_locations}개의 지점 번호를 입력하세요:")
    for _ in range(num_locations):
        index = int(input("지점 번호 입력: ")) - 1
        while index < 0 or index >= len(unique_locations):
            print("유효하지 않은 번호입니다. 다시 입력해주세요.")
            index = int(input("지점 번호 입력: ")) - 1
        selected_indices.append(index)

    # 선택한 지점들의 이름 확인
    selected_locations = [unique_locations[i] for i in selected_indices]

    # 차트를 그리기 위한 설정
    plt.figure(figsize=(10, 6))

    # 각 선택한 지점에 대해 데이터 추출 및 차트에 추가
    for location in selected_locations:
        selected_data = df[df['지점명'] == location].iloc[:12]  # 첫 12개 데이터만 가져옴
        plt.plot(selected_data['일시'], selected_data['월 미세먼지 농도(㎍/㎥)'], label=location)  # X축은 시간, Y축은 미세먼지 농도

    # 그래프 설정
    plt.title("선택한 지점의 미세먼지 농도 비교")
    plt.xlabel("측정시간")
    plt.ylabel("미세먼지 농도 (µg/m³)")
    plt.legend()  # 각 지점의 라벨 표시
    plt.xticks(rotation=45)  # 시간 라벨 회전
    plt.tight_layout()  # 레이아웃 조정

    # 그래프 출력
    plt.show()



In [28]:
# 미세먼지 농도에 따른 색상을 반환하는 함수
def get_pm_color(pm_value):
    if pm_value <= 30:
        return 'blue'  # 좋음
    elif 31 <= pm_value <= 80:
        return 'green'  # 보통
    elif 81 <= pm_value <= 150:
        return 'orange'  # 나쁨
    else:
        return 'red'  # 매우 나쁨

def plot_pm_map(pm_df, location_df):
    # 일시는 정확한 입력이 들어온다고 가정
    target_date = input("일시를 입력하세요 (형식: YYYY-MM, 예: '2023-11'): ")
    date = target_date.split("-")
    if date[0] not in ["2023", "2024"]:
        print("지원하지 않는 년도입니다")
        return None
    elif date[0] == "2023" and date[1] not in ["11", "12"] :
        print("지원하지 않는 월입니다.")
        return None
    elif date[1] not in [f"{num:02}" for num in range(1, 11)]:
        print("지원하지 않는 월입니다.")
        return None
    
         
    # 입력받은 '일시'에 해당하는 데이터를 필터링
    filtered_data = pm_df[pm_df['일시'] == target_date]

    # 기본 지도 설정 (서울을 중심으로 설정)
    map_center = [37.5665, 126.978]  # 서울 위경도
    folium_map = folium.Map(location=map_center, zoom_start=10)

    # 필터링된 데이터와 위경도 데이터를 합쳐서 지도에 마커를 추가
    for _, row in filtered_data.iterrows():
        station_id = row['지점']
        pm_value = row['월 미세먼지 농도(㎍/㎥)']

        # 해당 지점에 대한 위경도 정보 찾기
        location_data = location_df[location_df['지점'] == station_id]
        
        if not location_data.empty:
            latitude = location_data.iloc[0]['위도']
            longitude = location_data.iloc[0]['경도']

            # 마커 색상 결정
            marker_color = get_pm_color(pm_value)

            # 마커 추가 (마커 클릭 시 월 미세먼지 농도 표시)
            folium.Marker(
                location=[latitude, longitude],
                popup=folium.Popup(f"미세먼지 농도: {pm_value} ㎍/㎥", max_width=250, min_width=150),  # 팝업 창 크기 설정
                icon=folium.Icon(color=marker_color)
            ).add_to(folium_map)

    # 지도 객체 반환
    return folium_map



In [29]:
def main():
    # 한글 글씨체
    plt.rcParams['font.family'] = 'Noto Sans KR'

    try:
        # 미세먼지 데이터 프레임 가져오기
        dfPM = pd.read_csv('황사관측(PM10).csv')
    
        # 위경도 데이터 프레임 가져오기
        dfLO = pd.read_csv('지점별 위경도.csv')
    
        # 데이터가 잘 불러와졌는지 확인
        if not dfPM.empty and not dfLO.empty:
            print("데이터를 정상적으로 불러왔습니다.")
        else:
            print("데이터를 불러오는 데 문제가 발생했습니다. 파일을 확인해 주세요.")
    except FileNotFoundError as e:
        print(f"파일을 찾을 수 없습니다: {e}")
    except pd.errors.EmptyDataError:
        print("빈 파일이거나 읽을 수 없는 형식의 파일입니다.")
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")


    # 데이터 전처리 한 번만 수행
    df_clean = preprocess_data(dfPM)
    
    while(True):
        print("2023-09 ~ 2024-10 월별 미세먼지 데이터 제공 프로그램")
        print("1. 사용자 지정 지역별 그래프")
        print("2. 일시시별 미세먼지 농도 지도 출력")
        num = input("기능 선택(1, 2) >> ")

        if (num == "1"):
            drawGraph(df_clean)
        elif(num == "2"):# 지도 그리기
            map_object = plot_pm_map(dfPM, dfLO)
            display(map_object)
            # 지도 저장
            try:
                # 지도 저장
                map_object.save("pm_map.html")
                print("지도 파일이 'pm_map.html'로 정상적으로 저장되었습니다.")
            except Exception as e:
                print(f"지도 저장 중 오류가 발생했습니다: {e}")
            
        else:
            print("해당 번호의 기능은 제공하지 않습니다.")

        # 종료 여부
        k = input("계속하시겠습니까? (0입력시 종료)")
        if (k == "0"):
            print("종료하겠습니다.")
            break



In [33]:
if __name__ == '__main__':
    main()

데이터를 정상적으로 불러왔습니다.
2023-09 ~ 2024-10 월별 미세먼지 데이터 제공 프로그램
1. 사용자 지정 지역별 그래프
2. 일시시별 미세먼지 농도 지도 출력


기능 선택(1, 2) >>  2
일시를 입력하세요 (형식: YYYY-MM, 예: '2023-11'):  2024-04


지도 파일이 'pm_map.html'로 정상적으로 저장되었습니다.


계속하시겠습니까? (0입력시 종료) 0


종료하겠습니다.
